# Factory

> Implementing the Single Responsibility Principle / Separation of concerns.

Following the scenario from the previous lesson: once we get too many factory methods inside a class, it might make sense to move them out of the class and group them into a separate entity.

We will now create a new class called `PointFactory` that contains the factories we created previously.

In [1]:
# This cell contains the same code as the previous lesson, but the factories have been removed.

from enum import Enum
from math import *

class CoordinateSystem(Enum):
    CARTESIAN = 1
    POLAR = 2

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __str__(self):
        return f'x: {self.x}, y: {self.y}'
    

In [2]:
# And here we have our new Factory, with the factories that were previously inside Point.

class PointFactory:
    @staticmethod
    def new_cartesian_point(x, y):
        return Point(x, y)

    @staticmethod
    def new_polar_point(rho, theta):
        return Point(rho * sin(theta), rho * cos(theta))

The way to use our new Factory is very similar to the code we used previously, but we simply call the method inside the Factory class to create a polar point:

In [3]:
p = Point(2,3)
p2 = PointFactory.new_polar_point(1,2)
print(p, p2)

x: 2, y: 3 x: 0.9092974268256817, y: -0.4161468365471424


You may have noted that our Factory heavily depends on the initializer of the `Point` class, which may cause issues if the initializer for that class were to change. We could try to reduce this dependency by creating something like:

```python
class PointFactory:
    @staticmethod
    def new_cartesian_point(x, y):
        p = Point()
        p.x = x
        p,y = y
        return p
    
    #...
```

But this isn't a critical issue in this instance because there is an intrinsic relationship between the factory and the object that is actually getting constructed.

Now let's talk about discoverability: as it currently stands, the user may not be aware that the Factory exists because the user looks at the `Point` class and sees an initializer that already provides an instance, so the user may think that you can only initialize a Point with cartesian coordinates.

There isn't much we can do about this issue; we can always point out that the Factory exists in the documentation.

One more thing: we have defined our Factory as a separate class, but we could also define it as a class inside our `Point` class. This is typically done in other programming languages but it's not as common in Python; the initializer is always going to be public anyway because there is no way to make it private. But for the sake of completeness, let's move our Factory inside the base class:

In [4]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __str__(self):
        return f'x: {self.x}, y: {self.y}'

    class PointFactory:
        """
        We don't need to make the methods static anymore because the behavior doesn't change.
        However, we need to add back the 'self' as a param for our factory methods.
        """
        def new_cartesian_point(self, x, y): 
            return Point(x, y)

        def new_polar_point(self, rho, theta):
            return Point(rho * sin(theta), rho * cos(theta))

And we would now use it like this:

In [5]:
p = Point(2,3)
p2 = Point.PointFactory().new_polar_point(1,2) # note that we're calling Point first.
print(p, p2)

x: 2, y: 3 x: 0.9092974268256817, y: -0.4161468365471424


The main motivation to do this is in scenarios where our Factories store some sort of ***state***. In our example, maybe we want to create separate factories; a point factory which operates normally and a point factory with a bias value that gets added when initialized from polar, for example. This wouldn't be possible with static factory methods because we would need non static elements to perform the calculations.

If you don't want your factories static but you want `PointFactory` to act in a static fashion, we can add this at the end of the `Point` class:

In [6]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __str__(self):
        return f'x: {self.x}, y: {self.y}'

    class PointFactory:
        def new_cartesian_point(self, x, y): 
            return Point(x, y)

        def new_polar_point(self, rho, theta):
            return Point(rho * sin(theta), rho * cos(theta))
        
    # Add this
    factory = PointFactory()

Doing this makes it possible to access the Factory with `Point.factory`; essentially making `factory` a singleton instance of sorts of a point factory for our Point object.

In [7]:
p = Point(2,3)
p2 = Point.factory.new_polar_point(1,2) # calling our factory instance
print(p, p2)

x: 2, y: 3 x: 0.9092974268256817, y: -0.4161468365471424
